In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-07-20 21:02:14--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  58.1MB/s    in 0.4s    

2024-07-20 21:02:15 (58.1 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
# CLEAN UP THE DATA

duplicate_books = df_books.groupby(['title', 'author']).title.agg(['count']).reset_index().query('count > 1')
duplicates_books_count = duplicate_books['count'].sum() - len(duplicate_books)

duplicate_ratings = df_ratings.groupby(['isbn', 'user']).isbn.agg(['count']).reset_index().query('count > 1')
duplicates_ratings_count = duplicate_ratings['count'].sum() - len(duplicate_ratings)

## Modify the DF to drop duplicate or irrelevant rows
df_books = df_books.drop_duplicates(subset=['title', 'author'])
df_ratings = df_ratings.drop_duplicates(subset=['isbn', 'user'])

print("Found and removed {:,} duplicate copies of books and {:,} duplicate copies of ratings".format(duplicates_books_count, duplicates_ratings_count))

## Books
books_count_before = len(df_books)
books_with_ratings = df_books.merge(df_ratings, on='isbn')
grouped_by_isbn = books_with_ratings.groupby(['isbn', 'title']).rating.agg(['count', 'mean']).reset_index()
books_min_count = 100
acceptable_books = grouped_by_isbn.query('count >= {}'.format(books_min_count))['isbn'].tolist()
grouped_by_isbn = grouped_by_isbn[grouped_by_isbn['isbn'].isin(acceptable_books)]
df_books = df_books[df_books['isbn'].isin(acceptable_books)]
books_count_after = len(df_books)
b_percent_change = round((books_count_before-books_count_after)/books_count_before*100, 2)
print('Removed {:,} rows ({}%) of books with less than {} reviews'.format(books_count_before - books_count_after, b_percent_change, books_min_count))

## Users
users_count_before = len(df_ratings)
ratings_min_count = 200;
df_ratings = df_ratings[df_ratings['isbn'].isin(acceptable_books)]
acceptable_users = df_ratings.groupby(['user']).rating.agg(['count']).reset_index().query('count >= {}'.format(ratings_min_count))['user'].tolist()
df_ratings = df_ratings[df_ratings['user'].isin(acceptable_users)]
users_count_after = len(df_ratings)
u_percent_change = round((users_count_before-users_count_after)/users_count_before*100,2)
print('Removed {:,} rows ({}%) of user ratings with less than {} reviews per account or invalid books'.format(users_count_before - users_count_after, u_percent_change, ratings_min_count))


Found and removed 20,175 duplicate copies of books and 0 duplicate copies of ratings
Removed 250,596 rows (99.76%) of books with less than 100 reviews
Removed 1,146,574 rows (99.72%) of user ratings with less than 200 reviews per account or invalid books


In [5]:
# FORMAT THE LAST DATA BEFORE CREATING MODAL
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df = df_ratings
counts1 = df['user'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]


merged_df = pd.merge(right=df, left = df_books, on="isbn")
merged_df = merged_df.drop_duplicates(subset=["title", "user"])

books_features_pivot = merged_df.pivot(
  index='title',
  columns='user',
  values='rating'
).fillna(0)

mat_books_features = csr_matrix(books_features_pivot.values)

In [6]:
# function to return recommended books - this will be tested
## function to return recommended books - this will be tested
def get_recommends(book = "", n = 5):
  """
  make top n books recommendations
  Parameters
  ----------
  book: str, name of user input book
  n: int, top n recommendations
  """
  # Prepare for model
  pivot = books_features_pivot
  titles = list(pivot.index.values)
  data = pivot.values

  def title_2_index(title):
    ind = titles.index(title)
    return data[ind,:]

  def index_2_title(ind):
    return titles[ind]


  # Build model
  model = NearestNeighbors(metric="cosine",algorithm="brute", p=2)
  model.fit(data)

  # Run model to get recommendations
  idx = title_2_index(book)
  distances, indices = model.kneighbors(
    np.reshape(idx,[1,-1]),
    n_neighbors=n+1
  )

  raw_recommends = sorted(
    list(
      zip(
        indices.squeeze().tolist(),
        distances.squeeze().tolist()
      )
    ),
    key=lambda x: x[1]
  )[1:]

  # print results
  recommended_books = []
  print('Recommendations for {}:'.format(book))
  for i, (idx, dist) in enumerate(raw_recommends):
      dist = dist
      recommended_books.append([index_2_title(idx), dist])
      print('{0}: {1}, with distance of {2:,.2f}'.format(i+1, index_2_title(idx), dist))
  print('-----------------')
  return [book, recommended_books]

In [8]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))", 10)
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True', 'The Lovely Bones: A Novel']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77, 0.72]
  recommended_books.reverse()
  recommended_books_dist.reverse()

  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Recommendations for Where the Heart Is (Oprah's Book Club (Paperback)):
1: The Lovely Bones: A Novel, with distance of 0.72
2: I Know This Much Is True, with distance of 0.77
3: The Surgeon, with distance of 0.77
4: The Weight of Water, with distance of 0.77
5: I'll Be Seeing You, with distance of 0.80
6: The Dive From Clausen's Pier (Alex Awards), with distance of 0.80
7: Tis: A Memoir, with distance of 0.81
8: Icy Sparks, with distance of 0.81
9: Unspeakable, with distance of 0.81
10: What Looks Like Crazy On An Ordinary Day, with distance of 0.82
-----------------
You passed the challenge! 🎉🎉🎉🎉🎉
